<a href="https://colab.research.google.com/github/rodrigozago/compiladores/blob/master/Lexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analisador léxico
==================

Definição da tabela de simbolos
--------------------

Classe que define a tabela de simbolos.

symTable: Estrutura do tipo dict para armazenar os tokens:

```{ lexema<string> : token<Token> }```

In [ ]:
class SymTable:
    def __init__(self):
        self.symTable = dict();
    def addNewEntry(self, token):
        if(token.lexema in self.symTable):
            raise Exception("O lexema {} já foi adcionado à tabela de simbolos anteriormente".format(token.lexema))
        else:
            self.symTable[token.lexema] = token
    def getEntry(self, lexema):
        if(lexema in self.symTable):
            return self.symTable[lexema]
        else:
            return None

Definição da classe Token
--------------------------

In [ ]:
class Token:
  def __init__(self, lexema, token, tipo):
    self.lexema = lexema
    self.token = token
    self.tipo = tipo

  def __str__(self):
    return "Lexema: {} Token: {} Tipo: {}".format(self.lexema, self.token, self.tipo)

Adicionando palavras chave da linguagem na tabela de simbolos
---------------------------------------

In [ ]:
# Creating symbol table
table = SymTable()

# Add keywords into symbol table
try:
    table.addNewEntry(Token('inicio', 'inicio', '-'))
    table.addNewEntry(Token('varinicio', 'varinicio', '-'))
    table.addNewEntry(Token('varfim', 'varfim', '-'))
    table.addNewEntry(Token('escreva', 'escreva', '-'))
    table.addNewEntry(Token('leia', 'leia', '-'))
    table.addNewEntry(Token('se', 'se', '-'))
    table.addNewEntry(Token('entao', 'entao', '-'))
    table.addNewEntry(Token('fimse', 'fimse', '-'))
    table.addNewEntry(Token('fim', 'fim', '-'))
    table.addNewEntry(Token('inteiro', 'inteiro', '-'))
    table.addNewEntry(Token('lit', 'lit', '-'))
    table.addNewEntry(Token('real', 'real', '-'))    
except Exception as e:
    print("erro: {}".format(e))



In [ ]:
print(table.getEntry('inicio'))

Lexema: inicio Token: inicio Tipo: -


Definição do autômato finito
-----------------------------

Definindo função de transição

In [ ]:
tf = dict();
#NUM
tf[(0, 'a')] = 1;

In [ ]:
class DFA:
    current_state = None;
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states;
        self.alphabet = alphabet;
        self.transition_function = transition_function;
        self.start_state = start_state;
        self.accept_states = accept_states;
        self.current_state = start_state;
        return;
    
    def transition_to_state_with_input(self, input_value):
        if ((self.current_state, input_value) not in self.transition_function.keys()):
            # if has a '.'
            if((self.current_state, '.') in self.transition_function.keys()):
                self.current_state = self.transition_function[(self.current_state, '.')];
                return;
            self.current_state = None;
            return;
        self.current_state = self.transition_function[(self.current_state, input_value)];
        return;
    
    def in_accept_state(self):
        return self.current_state in accept_states;
    
    def go_to_initial_state(self):
        self.current_state = self.start_state;
        return;
    
    def run_with_input_list(self, input_list):
        self.go_to_initial_state();
        for inp in input_list:
            self.transition_to_state_with_input(inp);
            continue;
        return self.in_accept_state();
    pass;



states = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22};
# TODO - Wire all alphabet
alphabet = {'a', 'b', 'c', 'd'};

tf = dict();
tf[(0, 'a')] = 1;
tf[(0, 'b')] = 2;
tf[(0, 'c')] = 3;
tf[(0, 'd')] = 0;
tf[(1, 'a')] = 1;
tf[(1, 'b')] = 2;
tf[(1, 'c')] = 3;
tf[(1, 'd')] = 0;
tf[(2, 'a')] = 1;
tf[(2, 'b')] = 2;
tf[(2, 'c')] = 3;
tf[(2, 'd')] = 0;
tf[(3, 'a')] = 1;
tf[(3, 'b')] = 2;
tf[(3, 'c')] = 3;
tf[(3, 'd')] = 0;
start_state = 0;
accept_states = {2, 3};

d = DFA(states, alphabet, tf, start_state, accept_states);

inp_program = list('abcdabcdabcd');

print (d.run_with_input_list(inp_program));

Definição do Analisador Léxico
----------------------------------------

In [ ]:
class Scanner:
    def __init__():
        pass;
    pass;

Experimento com DFA
--------------

In [2]:
class DFA:
    current_state = None;
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states;
        self.alphabet = alphabet;
        self.transition_function = transition_function;
        self.start_state = start_state;
        self.accept_states = accept_states;
        self.current_state = start_state;
        return;
    
    def transition_to_state_with_input(self, input_value):
        print('debug: encontrei um {}'.format(input_value));
        # Verify if is .
        if((self.current_state, input_value) not in self.transition_function.keys()):
            if((self.current_state, '.') in self.transition_function.keys()):
                self.current_state = self.transition_function[(self.current_state, '.')];
                print('debug: casei com .');
                return;
            print('debug: não consegui casar');
            self.current_state = None;
            return;
        self.current_state = self.transition_function[(self.current_state, input_value)];
        return;
    
    def in_accept_state(self):
        return self.current_state in accept_states;
    
    def go_to_initial_state(self):
        self.current_state = self.start_state;
        return;
    
    def run_with_input_list(self, input_list):
        self.go_to_initial_state();
        for inp in input_list:
            self.transition_to_state_with_input(inp);
            continue;
        return self.in_accept_state();
    pass;



states = {0, 1};
alphabet = {L, D, OP, ' ','{','}','"',';','(',')','.', '>','<','='};
L = {'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','y','x','z'};
D = {'0','1','2','3','4','5','6','7','8','9'};
OP = {'+','-','*','/'};

tf = dict();
tf[(0, " ")] = 0;
tf[(0, "\n")] = 0;
tf[(0, "\t")] = 0;
tf[(0, D)] = 1;
tf[(1, '.')] = 2;
tf[(1, 'E')] = 4;
tf[(1, 'e')] = 4;
tf[(2, D)] = 3;
tf[(2, "QUALQUER COISA")] = 23;
tf[(3, D)] = 3;
tf[(3, 'E')] = 4;
tf[(3, 'e')] = 4;
tf[(4, '+')] = 5;
tf[(4, '-')] = 5;
tf[(4, D)] = 6;
tf[(4, 'QUALQUER COISA')] = 23;
tf[(5, D)] = 6;
tf[(5, 'QUALQUER COISA')] = 23;
tf[(6, D)] = 6;
tf[(0, '"')] = 7;
tf[(7, 'QUALQUER COISA')] = 7;
tf[(7, '"')] = 8;
tf[(0, L)] = 9;
tf[(9, L)] = 9;
tf[(9, D)] = 9;
tf[(9, ' ')] = 9;
tf[(0, '{')] = 10;
tf[(10, 'QUALQUER COISA')] = 10;
tf[(10, '}')] = 11;
tf[(0, EOF)] = 12;
tf[(0, '>')] = 13;
tf[(13, '=')] = 14;
tf[(0, '<')] = 15;
tf[(15, '=')] = 14;
tf[(15, '>')] = 16;
tf[(15, 'QUALQUER COISA')] = 17;
tf[(0, '=')] = 18;
tf[(0, OP)] = 19;
tf[(0, '(')] = 20;
tf[(0, ')')] = 21;
tf[(0, ';')] = 22;


start_state = 0;
accept_states = {0};

d = DFA(states, alphabet, tf, start_state, accept_states);

inp_program = list('aca');

print (d.run_with_input_list(inp_program));

NameError: ignored